## Support Vector Machines
* Poderosa metodologia para resolver problemas de aprendizagem de máquina.
* Proposto em 1995 pelo russo Vladimir Vapnik.
* Muito utilizado atualmente em diversos tipos de aplicações.
* O problema dos problemas não linearmente separáveis 
* O problema dos problemas não linearmente separáveis


![](https://i.ibb.co/tsDTKzn/SVM-Picture1.png)

* Problema não linearmente separável

### O processo de classificação:
  + Entendendo o parâmetro C, Soft Margin: 
    - A aplicação de um método puramente linear para classificar um conjunto de dados pode sofrer com dois problemas bastante comuns:
      - Outliers
      - Exemplos rotulados erroneamente
  + Mesmo assim o SVM ainda assim pode ser aplicado através do uso do parâmetro C (soft margin - variáveis de folga)
#### O truque do Kernel:
    - Realiza a projeção das características em um espaço multidimencional, maior que o vetor de entrada onde as classes podem ser separadas por um hiperplano.

![](https://i.ibb.co/n19dmvq/SVM-Picture2.png)

#### Vantagens:
  + Considerado o estado da arte dos classificadores.
  + Trabalha com problemas linearmente e não-linearmente separáveis.
  + Desempenho superior a maioria dos classificadores tradicionais.
  
#### Desvantagens:
  + A calibragem dos parâmetros envolvidos e escolha do tipo de kernel não é uma tarefa trivial.
  + O tempo de treinamento pode ser extremamente lento e computacionalmente complexo dependendo da quantidade de características.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt 

from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn import svm
from sklearn.preprocessing import LabelEncoder

# Importe as bibliotecas de Pipelines e Pré-processadores
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Importa o pacote OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

import sys

## 1) Importando os dados já tratados sem OneHotEncoding:

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !ls "/content/drive/My Drive/datasets"
# data = pd.read_csv('/content/drive/My Drive/datasets/dados_clust_v2.csv',sep=',') # faz a leitura dos dados

data = pd.read_csv("/Users/andrerodrigues/Google Drive/FIAP-MBA 8IA/006 - Modelos de IA e ML/Trabalho_Final_2/Exercicio8_v2/dados_clust_v2.csv", sep =",")

data = data.iloc[:,1:]
data.head()

## 2) Importando os dados já tratados com OneHotEncoding:

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !ls "/content/drive/My Drive/datasets"
# data_ohe = pd.read_csv('/content/drive/My Drive/datasets/data_ohe_nostd_v2.csv',sep=',') # faz a leitura dos dados

data_ohe = pd.read_csv("/Users/andrerodrigues/Google Drive/FIAP-MBA 8IA/006 - Modelos de IA e ML/Trabalho_Final_2/Exercicio8_v2/data_ohe_nostd_v2.csv", sep =",")

data_ohe = data_ohe.iloc[:,1:]
data_ohe["dbscan"] = data["dbscan"]
data_ohe["kmeans"] = data["kmeans"]
data_ohe["som"] = data["som"]
data_ohe.head()

#### a) Apenas como curiosidade, vejamos como as classes estão distribuidas em 'dbscan', 'kmeans' e 'som'

In [ ]:
plt.figure(figsize=(10,7))
plot1 = sb.countplot(x='dbscan',data=data, palette='hls')
plt.title("Verificando classes de dbscan")
plt.show(plot1)
plt.figure(figsize=(10,7))
plot2 = sb.countplot(x='kmeans',data=data, palette='hls')
plt.title("Verificando classes de kmeans")
plt.show(plot2)
plt.figure(figsize=(10,7))
plot3 = sb.countplot(x='som',data=data, palette='hls')
plt.title("Verificando classes de som")
plt.show(plot3)

## 3) Seleciona o dataset para ser tratado pelo Modelo de Machine Learning e Separa as Classes

In [ ]:
import traceback

def dataset_for_machine(dataset, target):
    
    # Tente receber o nome do arquivo
    try:
        d = dataset
        t = target
        
        
    
        if d == 'data':
            #No caso do DBSCAN, as amostras cujo o cluster for igual a 0, são consideradas outliers e não podemos considera-las como classe do problema.
            if t == 'dbscan':
                data.drop(data[data.dbscan == 0].index, inplace=True)
            # Troca o nome do dataset para df. A partir desse ponto, o dataset para ser tratado pelo modelo de ML será conhecido por df.
            df = data
            # Separa a classe
            classes = df[t]
            df.drop(columns=['dbscan','kmeans', 'som'], axis=1, inplace=True)


        elif d == 'data_ohe':
            #No caso do DBSCAN, as amostras cujo o cluster for igual a 0, são consideradas outliers e não podemos considera-las como classe do problema.
            if t == 'dbscan':
                data.drop(data[data.dbscan == 0].index, inplace=True)
            # Troca o nome do dataset para df. A partir desse ponto, o dataset para ser tratado pelo modelo de ML será conhecido por df.
            df = data_ohe
            # Separa a classe
            classes = df[t]
            df.drop(columns=['dbscan','kmeans', 'som'], axis=1, inplace=True)

        else:
            print("Escolha entre 'data' ou 'data_ohe'")
            exit

    # Se ocorrer um erro
    except:
        # Mostre na tela
        trace = traceback.format_exc()
        print ('Aconteceu um erro:\n', trace)
        # Encerre o programa
        raise SystemExit
        
    return df, classes
    
# df, classes = dataset_for_machine(input("Entre com o dataset que será tratado pela maquina('data' ou 'data_ohe') e o target('kmeans' ou 'som') separados por ',':"))
dataset = input("Entre com o dataset que será tratado pela maquina('data' ou 'data_ohe'):")
target = input("Entre com o target que será tratado pela maquina ('dbscan','kmeans' ou 'som'):")
df, classes = dataset_for_machine(dataset, target)

* A partir desse ponto, o dataset para ser tratado pelo modelo de ML será conhecido por df. A seleção se ele vai ser passado com ou sem One Hot Encoding foi feita acima.

In [ ]:
df.head()

In [ ]:
classes

## 4) Faz a validação via crossvalidation (k-fold)

In [ ]:
def Acuracia(clf,X,y):
    resultados = cross_val_predict(clf, X, y, cv=5)
    return metrics.accuracy_score(y,resultados)

## 5) Pre-processamento de dados

In [ ]:
# def remove_features(lista_features):
#     for i in lista_features:
#         data.drop(i, axis=1, inplace=True)
#     return 0

## 9) Support Vector Machines 
#### a) Preparando os pipes com as configurações do parâmetro C, Kernel e Normalização dos dados.

In [ ]:
pip_1 = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', svm.SVC())
])

pip_2 = Pipeline([
    ('min_max_scaler', MinMaxScaler()),
    ('clf', svm.SVC())
])

pip_3 = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', svm.SVC(kernel='rbf'))
])

pip_4 = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', svm.SVC(kernel='poly'))
])

pip_5 = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', svm.SVC(kernel='linear'))
])

#### b) Teste com apenas LabelEncoder na coluna 'classes' usando o pipeline 'pip_1'

In [ ]:
Acuracia(pip_1,df,classes)

#### c) Teste com apenas LabelEncoder na coluna 'classes' usando o pipeline 'pip_2'

In [ ]:
Acuracia(pip_2,df,classes)

#### d) Testando Kernels 
##### Kernel rbf

In [ ]:
Acuracia(pip_3,df,classes)

##### Kernel Polynomial

In [ ]:
Acuracia(pip_4,df,classes)

##### Kernel Linear

In [ ]:
Acuracia(pip_5,df,classes)

## 9) GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV

lista_C = [0.001, 0.01, 0.1, 1, 10,100]
lista_gamma = [0.001, 0.01, 0.1, 1, 10, 100]

In [ ]:
parametros_grid = dict(clf__C=lista_C, clf__gamma=lista_gamma)

#### a) Faz o tuning dos parametros testando cada combinação utilziando CrossValidation com 10 folds e analisando a acurácia

In [ ]:
grid = GridSearchCV(pip_3, parametros_grid, cv=10, scoring='accuracy')

In [ ]:
grid.fit(df,classes)

In [ ]:
grid.cv_results_

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

#### b) Métricas de Avaliação de Modelos

In [ ]:
pip_6 = Pipeline([
('scaler',StandardScaler()),
('clf', svm.SVC(kernel='rbf',C=1,gamma=1))
])

In [ ]:
resultados = cross_val_predict(pip_6, df, classes, cv=10)

In [ ]:
c_names = [str(i) for i in np.unique(classes)]
print (metrics.classification_report(classes,resultados,target_names=c_names))